In [2]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM

ckpt = "google/gemma-3-1b-it"
model = Gemma3ForCausalLM.from_pretrained(
    ckpt, torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(ckpt)

In [3]:
ocr_text = "합격을 서울대 의대 신OO 서울대 체육교육과 박O0 축하합니다!! 서운대의대2년인아진학 학교이름은 및내쥐서 고마워요 고통화교 동창회일동"
no_info = "정보를 찾을 수 없습니다."

In [4]:
messages = [
    [
        {
            "role": "system",
            "content":[
                {
                    "type": "text",
                    "text": "You are an expert in analyzing the purpose of text. Your task is to classify the given text into the most relevant category and determine its legality based on strict criteria."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""
                                Analyze the meaning of the following banner text and classify it into the most relevant category from the list below.

                                **Banner Text:** {ocr_text}

                                ### Categories:
                                1. Politics
                                2. Public interest
                                3. Commercial purposes
                                4. Other

                                ### Judgment Criteria:
                                - If the category is Politics, Public interest then:
                                  **Judgment: "legal"**
                                
                                - If the category is **Commercial purposes or Other**, then:
                                  **Judgment: "illegal"**
                                

                                ### Output Format (Strictly Follow This):

                                ```
                                Category: "Politics"
                                Judgment: "legal"
                                ```
                            """
                            
                }
            ]
            
        }
    ],
]

inputs = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device)

input_len = inputs["input_ids"].shape[-1]

generation = model.generate(**inputs, max_new_tokens=200, do_sample=False)
generation = generation[0][input_len:]

decoded = tokenizer.decode(generation, skip_sepcial_tokens=True)
print(decoded)

/opt/homebrew/anaconda3/envs/banner_it/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/homebrew/anaconda3/envs/banner_it/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


```
Category: "Other"
Judgment: "illegal"
```
<end_of_turn>


In [ ]:
messages = [
    [
        {
            "role": "system",
            "content":[
                {
                    "type": "text",
                    "text": "You are a helpful assistant who extract phone numbers and company name from text extracted from banners."
                }
            ]
        },
        {
            "role": "user",
            "content":[
                {
                    "type": "text",
                    "text": f"""
                                Extract phone numbers and a company name or store name from {ocr_text}. If there are none, answer {no_info}.
                                The output is printed as an example.
                                
                            """
                }
            ]
        },
    ],
]

inputs = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device)

input_len = inputs["input_ids"].shape[-1]

generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
generation = generation[0][input_len:]

decoded = tokenizer.decode(generation, skip_sepcial_tokens=True)
print(decoded)

Okay, here's the extraction based on the provided text:

*   **Phone Number:** (Not present in the text)
*   **Company Name:** 서운대 (Suwon Da)

**Explanation:**

The text mentions "서운대의대2년인아진학 학교이름은 및내쥐서" which translates to "The name of the school 'A Jin Hak' is and more."  This strongly suggests a school affiliated with Seoul National University.
